# Electronic Hamiltonian Simulation with Strawberry Fields

The simulation of atoms, molecules and other biochemical systems is another application uniquely suited to quantum computation. For example, the ground state energy of large systems, the dynamical behaviour of an ensemble of molecules, or complex molecular behaviour such as protein folding, are often computationally hard or downright impossible to determine via classical computation or experimentation.

In the discrete-variable qubit model, efficient methods of Hamiltonian simulation have been discussed at-length, providing several implementations depending on properties of the Hamiltonian, and resulting in a linear simulation time. Efficient implementations of Hamiltonian simulation also exist in the continuous variable model; with specific application to *Bose-Hubbard Hamiltonians* (describing a system of interacting bosonic particles on a lattice of orthogonal position states), as such this is a method which is perfectly suited to photonic quantum computation.

In [1]:
import numpy as np
import strawberryfields as sf
from strawberryfields.ops import *

### Implementation

In [2]:
np.random.seed(42)

hamiltonian_simulation = sf.Program(2)

In [3]:
# set the Hamiltonian parameters
J = 1           # hopping transition
U = 1.5         # on-site interaction
k = 20          # Lie product decomposition terms
t = 1.086       # timestep
theta = -J*t/k
r = -U*t/(2*k)

In [4]:
with hamiltonian_simulation.context as q:
    # prepare the initial state
    Fock(2) | q[0]

    # Two node tight-binding
    # Hamiltonian simulation

    for i in range(k):
        BSgate(theta, np.pi/2) | (q[0], q[1])
        Kgate(r)  | q[0]
        Rgate(-r) | q[0]
        Kgate(r)  | q[1]
        Rgate(-r) | q[1]

In [6]:
sf.ping()

You have successfully authenticated to the platform!


In [7]:
eng = sf.Engine(backend="fock", backend_options={"cutoff_dim": 5})

In [9]:
results = eng.run(hamiltonian_simulation)

In [10]:
state = results.state
print("P(|0, 2>) = ", state.fock_prob([0, 2]))
print("P(|1, 1>) = ", state.fock_prob([1, 1]))
print("P(|2, 0>) = ", state.fock_prob([2, 0]))

P(|0, 2>) =  0.5224012457200213
P(|1, 1>) =  0.23565287685672467
P(|2, 0>) =  0.24194587742325993


In [11]:
result = [state.fock_prob([0, 2]), state.fock_prob([1, 1]), state.fock_prob([2, 0])]
print(np.sum(result))

1.000000000000006


In [12]:
from scipy.linalg import expm

H = J*np.sqrt(2)*np.array([[0,1,0],[1,0,1],[0,1,0]]) + U*np.diag([1,0,1])
init_state = np.array([0,0,1])

theoretical_result = np.abs(np.dot(expm(-1j*t*H), init_state))**2
print(theoretical_result)

[0.52249102 0.23516277 0.24234621]


In [13]:
print(np.all(np.abs(theoretical_result - result) < 1e-2))

True
